<img align = 'center' src="../../Images/ML_SWE.jpg" alt = '.../Images/' width = '1000'/>

## Prediction Simulation for Water Year 2019

We will be testing and evaluating the performance of the model over the 2019 WY at select locations

For example, the operational capacity of the model should begin at the beginning of the water year (WY week 1) with a previous SWE value of 0-in for each 1-km grid.
Model evaluation could then assess the forecasting skill on a hold-out dataset of an entire year in which known neighboring values do not control predictions, but rather driving to-date predictions from the predicted previous SWE values. 
Due to the high model performance, future work will target the assessment of model skill with an operational motivation

## Model Training/Testing influence on Model Results.

The model training/testing partitioning methodology has a strong influence on model performance and the goal of model evaluation.
The objective of the modeling effort was to examine the spatial extrapolation capacity of the model from selected monitoring stations to the overall region, best suited to a 75/25% training/testing split, respectively.
While it is critical to address the strong serial correlation in SWE accumulation and melt throughout the season, the high correlation between weeks has the potential to inflate model skill when using a 75/25% training/testing split due to the previous SWE feature being known.
An assessment of the operational capacity of the model is different than assessing the ability to extrapolate regional SWE from in-situ monitoring stations.
For example, the operational capacity of the model should begin at the beginning of the water year (WY week 1) with a previous SWE value of 0-in for each 1-km grid.
Model evaluation could then assess the forecasting skill on a hold-out dataset of an entire year in which known neighboring values do not control predictions, but rather driving to-date predictions from the predicted previous SWE values. 
Due to the high model performance, future work will target the assessment of model skill with an operational motivation.

In [1]:
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '..') #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import Hindcast_Initialization, NSM_SCA
import warnings
warnings.filterwarnings('ignore')

#Set working directories
cwd = os.getcwd()
datapath = f"{os.path.expanduser('~')}/SWEML" 

2024-04-23 15:14:17.606517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-23 15:14:17.606541: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Bring prediction DFs in
The Prediction DFs do not store after a git push and this will save your results, make sure you have the correct model name in the function

In [ ]:
# frequency = 'Daily'
# fSCA = True
# modelname = "Neural_Network"
# Hindcast_Initialization.AWS_to_Hindcast(modelname, frequency, fSCA)

# Initiate Simulation

You can run all regions or just ones for development, not that only the regions entered below will work for the Hindcast Evaluator

## Run your SSM in hindcast mode to evaluate operational capacity.

In [2]:
#To create .netrc file
#import earthaccess
#earthaccess.login(persist=True)

'''
machine urs.earthdata.nasa.gov
	login username
	password password
	account email
'''

#Set working directories
cwd = os.getcwd()
datapath = f"{os.path.expanduser('~')}/SWEML" 

new_year = '2022'
threshold = '10.0' #This threshold is standardized for now, to recalculate see Dr. Johnson
Region_list = [ 'N_Sierras', 'S_Sierras_High', 'S_Sierras_Low','Greater_Yellowstone', 
    'N_Co_Rockies', 'SW_Mont', 'SW_Co_Rockies', 'GBasin', 'N_Wasatch', 'N_Cascade',
     'S_Wasatch', 'SW_Mtns', 'E_WA_N_Id_W_Mont', 'S_Wyoming', 'SE_Co_Rockies', 
     'Sawtooth', 'Ca_Coast', 'E_Or', 'N_Yellowstone', 'S_Cascade', 'Wa_Coast',
     'Greater_Glacier', 'Or_Coast'
              ]

fSCA = True

frequency = 'Daily' #enter Weekly or Daily

datelist = Hindcast_Initialization.Hindcast_Initialization(cwd, datapath, new_year, threshold, Region_list, frequency, fSCA = fSCA)

Creating files for a historical simulation within  'N_Sierras', 'S_Sierras_High', 'S_Sierras_Low', 'Greater_Yellowstone', 'N_Co_Rockies', 'SW_Mont', 'SW_Co_Rockies', 'GBasin', 'N_Wasatch', 'N_Cascade', 'S_Wasatch', 'SW_Mtns', 'E_WA_N_Id_W_Mont', 'S_Wyoming', 'SE_Co_Rockies', 'Sawtooth', 'Ca_Coast', 'E_Or', 'N_Yellowstone', 'S_Cascade', 'Wa_Coast', 'Greater_Glacier', 'Or_Coast'  regions for water year  2022


In [4]:
#run the model through all time (data acqusition already completed)
model = 'Neural_Network'
frequency = 'Daily'
threshold = 10
NewSim = True

for day in tqdm(datelist):
    print(day)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(day, threshold=threshold, Regions = Region_list, modelname = model, frequency = frequency, fSCA =  fSCA, NewSim = NewSim)
    
    #Go get SNOTEL observations - all data currently loaded, set to True to download
    Snow.Get_Monitoring_Data_Threaded(getdata = True)

    #Initialize/Download the granules, all data preprocessed for the SSM activRegion_listhange to True to use the functions.
    Snow.initializeGranules(getdata = True)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing(SCA = True)

    #Agument with SCA
    Snow.augmentPredictionDFs()
 
    #Make predictions, set NewSim to False
     #Look to multiprocess, each region can do a prediction to speed things up. set NewSim to true for New simulation, turn to false once all data has been proces and saved.
    Snow.SWE_Predict(NewSim = NewSim, Corrections = False, fSCA = fSCA)


  0%|          | 0/268 [00:00<?, ?it/s]

Getting California Data Exchange Center SWE data from sites
Getting NRCS SNOTEL SWE data from sites
Getting VIIRS fSCA data and calculating the spatial average NSDI
VIIRS fSCA files found locally


100%|██████████| 3349/3349 [00:00<00:00, 8484.64it/s]


Regional data QA/QC


100%|██████████| 22/22 [00:15<00:00,  1.38it/s]


Calculating mean SCA for each geometry in each region...


100%|██████████| 23/23 [00:34<00:00,  1.51s/it]
2024-04-23 15:16:19.098846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-23 15:16:19.098875: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-23 15:16:19.098893: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (compute004.ual-ciroh.cluster): /proc/driver/nvidia/version does not exist
2024-04-23 15:16:19.099147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


5 out of the last 57 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f7105411820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


6 out of the last 58 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f716364cd30> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
  0%|          | 1/268 [01:26<6:26:13, 86.79s/it]

No snow in region:  Or_Coast
Getting California Data Exchange Center SWE data from sites
Getting NRCS SNOTEL SWE data from sites


### Model Hindcast complete

Lets see how your model performs within SSWEET in the [Hindcast_Evaluation](./Hindcast_Evaluation.ipynb) notebook.

In [3]:
modelname= 'Neural_Network'
folderpath = 'Predictions/Hold_Out_Year/Daily/fSCA_True/'
AWSpath = f"Hold_Out_Year/Daily/"
type = '.h5'
Hindcast_Initialization.Hindcast_to_AWS(modelname, folderpath, AWSpath, type)

['submission_format.h5', '2019_predictions.h5', '2022_predictions.h5', 'RegionWYTest.h5']
Pushing files to AWS


100% |########################################################################|
